In [1]:
from pathlib import Path

DATA_DIR = Path("/home/carlyn.1/dna-trait-analysis/data")

melpomene_reference_info_file = DATA_DIR / "Hmel2.5.gff3"

In [6]:
%%bash
/home/carlyn.1/dna-trait-analysis/tmp/mummer-4.0.1/nucmer -c 100 -p nucmer /home/carlyn.1/dna-trait-analysis/data/Heliconius_melpomene_melpomene_Hmel2.5.scaffolds.fa /home/carlyn.1/dna-trait-analysis/data/Heliconius_erato_demophoon_v1_-_scaffolds.fa

In [7]:
%%bash
/home/carlyn.1/dna-trait-analysis/tmp/mummer-4.0.1/show-coords -r -c -l nucmer.delta > nucmer.coords

In [8]:
%%bash
/home/carlyn.1/dna-trait-analysis/tmp/mummer-4.0.1/show-snps -C nucmer.delta > nucmer.snps

In [9]:
%%bash
/home/carlyn.1/dna-trait-analysis/tmp/mummer-4.0.1/show-tiling nucmer.delta > nucmer.tiling

In [74]:
rows = []
with open("nucmer.coords", "r") as f:
    for line in f.readlines()[5:]:
        # Seems to be inclusive (the start and end positions)
        (
            mel_start,
            mel_end,
            _,
            era_start,
            era_end,
            _,
            mel_len,
            era_len,
            _,
            idy,
            _,
            len_r,
            len_q,
            _,
            cov_r,
            cov_q,
            _,
            mel_tag,
            era_tag,
        ) = line.split()

        erato_chrom_info = era_tag.replace("Herato", "")
        if erato_chrom_info in ["_mt"]:
            continue
        erato_chrom_num = int(erato_chrom_info[:2])
        erato_scaf_num = int(erato_chrom_info[2:])
        melpomene_chrom_info = mel_tag.replace("Hmel2", "")
        melpomene_chrom_num = int(melpomene_chrom_info[:2])
        melpomene_scaf_num = int(melpomene_chrom_info[2:-1])

        real_mel_start = min(mel_start, mel_end)
        real_mel_end = max(mel_start, mel_end)
        real_era_start = min(era_start, era_end)
        real_era_end = max(era_start, era_end)

        rows.append(
            [
                real_mel_start,
                real_mel_end,
                real_era_start,
                real_era_end,
                mel_len,
                era_len,
                idy,
                len_r,
                len_q,
                cov_r,
                cov_q,
                mel_tag,
                era_tag,
                melpomene_chrom_num,
                melpomene_scaf_num,
                erato_chrom_num,
                erato_scaf_num,
            ]
        )

In [75]:
import pandas as pd

alignment_coords_df = pd.DataFrame(
    rows,
    columns=[
        "melpomene_start",
        "melpomene_end",
        "erato_start",
        "erato_end",
        "melpomene_length",
        "erato_length",
        "idy",
        "len_r",
        "len_q",
        "cov_r",
        "cov_q",
        "melpomene_tag",
        "erato_tag",
        "melpomene_chromosome",
        "melpomene_scaffold",
        "erato_chromosome",
        "erato_scaffold",
    ],
)

alignment_coords_df = alignment_coords_df[
    alignment_coords_df.melpomene_chromosome == alignment_coords_df.erato_chromosome
]  # Only keep alignments on the same chromosome
alignment_coords_df = alignment_coords_df.sort_values(
    by=["melpomene_chromosome", "melpomene_scaffold", "melpomene_start"]
)
alignment_coords_df.head()

,melpomene_start,melpomene_end,erato_start,erato_end,melpomene_length,erato_length,idy,len_r,len_q,cov_r,cov_q,melpomene_tag,erato_tag,melpomene_chromosome,melpomene_scaffold,erato_chromosome,erato_scaffold
8004,10000142,9998677,13297916,13299351,1466,1436,83.47,17206585,22325789,0.01,0.01,Hmel201001o,Herato0101,1,1,1,1
8005,10000267,10000572,13299872,13300176,306,305,88.75,17206585,22325789,0.00,0.00,Hmel201001o,Herato0101,1,1,1,1
8006,10000953,10002162,13300580,13301816,1210,1237,85.26,17206585,22325789,0.01,0.01,Hmel201001o,Herato0101,1,1,1,1
8007,10003128,10004074,13302673,13303589,947,917,84.12,17206585,22325789,0.01,0.00,Hmel201001o,Herato0101,1,1,1,1
8008,10006179,10006473,13306123,13306431,295,309,89.07,17206585,22325789,0.00,0.00,Hmel201001o,Herato0101,1,1,1,1


In [76]:
alignment_coords_df.shape

(158208, 17)

In [77]:
import pandas as pd

rows = []
other_names = ["chromosome", "scaffold"]
att_names = [
    "Dbxref",
    "ID",
    "Name",
    "Note",
    "Parent",
    "deletions",
    "description",
    "frameshifts",
    "stable_id",
    "symbol",
    "translation_stable_id",
]
with open(melpomene_reference_info_file, "r") as f:
    for line in f.readlines():
        cols = line.split("\t")
        if len(cols) == 1:
            continue  # Just a seperator line

        seqid, source, feat_type, start, end, score, strand, phase, attributes = cols
        atts = attributes.strip().split(";")
        att_cols = ["" for _ in range(len(att_names))]
        for att in atts:
            parts = att.split("=")
            key = parts[0]
            if key not in att_names:
                continue
            value = parts[1]
            att_idx = att_names.index(key)
            att_cols[att_idx] = value

            chrom_info = seqid.replace("Hmel2", "")
            chrom_num = int(chrom_info[:2])
            scaf_num = int(chrom_info[2:-1])

        other_cols = [chrom_num, scaf_num]
        rows.append(cols + other_cols + att_cols)

melpomene_reference_gene_info_df = pd.DataFrame(
    rows,
    columns=[
        "seqid",
        "source",
        "feat_type",
        "start",
        "end",
        "score",
        "strand",
        "phase",
        "attributes",
    ]
    + other_names
    + att_names,
)

In [78]:
melpomene_reference_gene_info_df.head()

,seqid,source,feat_type,start,end,score,strand,phase,attributes,chromosome,...,ID,Name,Note,Parent,deletions,description,frameshifts,stable_id,symbol,translation_stable_id
0,Hmel200067o,HGC,gene,69,6264,.,-,.,ID=HMEL005427;description=Nitric oxide synthas...,0,...,HMEL005427,,,,,Nitric oxide synthase like B,,HMEL005427,NOS-B,
1,Hmel200067o,HGC,mRNA,69,6264,.,-,.,"Dbxref=goslim_goa:GO:0003674,goslim_goa:GO:000...",0,...,HMEL005427-RA,,,HMEL005427,,,,HMEL005427-RA,,HMEL005427-RA
2,Hmel200067o,HGC,CDS,69,160,.,-,2,ID=CDS___0;Parent=HMEL005427-RA\n,0,...,CDS___0,,,HMEL005427-RA,,,,,,
3,Hmel200067o,HGC,CDS,2806,3016,.,-,0,ID=CDS___0;Parent=HMEL005427-RA\n,0,...,CDS___0,,,HMEL005427-RA,,,,,,
4,Hmel200067o,HGC,CDS,3238,3407,.,-,2,ID=CDS___0;Parent=HMEL005427-RA\n,0,...,CDS___0,,,HMEL005427-RA,,,,,,


In [79]:
gene_info_df[(gene_info_df.chromosome == 18) & (gene_info_df.symbol == "Optix")]

,seqid,source,feat_type,start,end,score,strand,phase,attributes,chromosome,...,ID,Name,Note,Parent,deletions,description,frameshifts,stable_id,symbol,translation_stable_id
4717,Hmel218003o,HGC,gene,705604,706407,.,+,.,ID=HMEL001028;description=Optix;stable_id=HMEL...,18,...,HMEL001028,,,,,Optix,,HMEL001028,Optix,


In [ ]:
# import numpy as np
#
# chrom_idx = (alignment_coords_df.melpomene_chromosome == 18)
# gene_start = 705604
# gene_end = 706407
# start_in_idx = (gene_start <= alignment_coords_df.melpomene_end.astype(int)) & (gene_start >= alignment_coords_df.melpomene_start.astype(int))
# end_in_idx = (gene_end <= alignment_coords_df.melpomene_end.astype(int)) & (gene_end >= alignment_coords_df.melpomene_start.astype(int))
#
# alignment_coords_df[chrom_idx & (start_in_idx | end_in_idx)]

,melpomene_start,melpomene_end,erato_start,erato_end,melpomene_length,erato_length,idy,len_r,len_q,cov_r,cov_q,melpomene_tag,erato_tag,melpomene_chromosome,melpomene_scaffold,erato_chromosome,erato_scaffold
133635,704417,707238,1249225,1252032,2822,2808,90.63,16450716,5168646,0.02,0.05,Hmel218003o,Herato1801,18,3,18,1


In [ ]:
gene_info_df = melpomene_reference_gene_info_df[
    melpomene_reference_gene_info_df.feat_type == "gene"
]

new_gene_info_rows = []
for idx, df_row in gene_info_df.iterrows():
    melpomene_start = int(df_row.start)
    melpomene_end = int(df_row.end)
    melpomene_chromosome = int(df_row.chromosome)
    melpomene_scaffold = int(df_row.scaffold)
    gene_id = df_row.ID
    gene_symbol = df_row.symbol

    # Find intersections
    filtered_alignment_df = alignment_coords_df[
        alignment_coords_df.melpomene_chromosome == melpomene_chromosome
    ]

    gene_start = int(df_row.start)
    gene_end = int(df_row.end)
    start_in_idx = (gene_start <= filtered_alignment_df.melpomene_end.astype(int)) & (
        gene_start >= filtered_alignment_df.melpomene_start.astype(int)
    )
    end_in_idx = (gene_end <= filtered_alignment_df.melpomene_end.astype(int)) & (
        gene_end >= filtered_alignment_df.melpomene_start.astype(int)
    )

    intersections = start_in_idx | end_in_idx

    filtered_alignment_df = filtered_alignment_df[intersections]

    erato_start = -1
    erato_end = -1
    erato_chromosome = -1
    erato_scaffold = -1
    iou_score = 0
    containment_score = 0
    if filtered_alignment_df.shape[0] > 0:
        min_start = int(filtered_alignment_df.melpomene_start.min())
        max_end = int(filtered_alignment_df.melpomene_end.max())

        # may have to incorporate the idy column from the aligment scores in these
        intersection = max(
            0, min(melpomene_end, max_end) - max(melpomene_start, min_start)
        )
        union = max(1, max(melpomene_end, max_end) - min(melpomene_start, min_start))

        if intersection == 0:
            continue

        iou_score = intersection / union
        containment_score = intersection / max(1, max_end - min_start)
        erato_start = int(filtered_alignment_df.erato_start.min())
        erato_end = int(filtered_alignment_df.erato_end.max())

        erato_chromosome = int(
            filtered_alignment_df.erato_chromosome.max()
        )  # Should be unique

        erato_scaffold = int(
            filtered_alignment_df.erato_scaffold.max()
        )  # May not be unique...so don't count on this

    new_gene_info_rows.append(
        [
            gene_id,
            gene_symbol,
            melpomene_chromosome,
            melpomene_scaffold,
            melpomene_start,
            melpomene_end,
            erato_chromosome,
            erato_scaffold,
            erato_start,
            erato_end,
            iou_score,
            containment_score,
        ]
    )

gene_aligment_info_df = pd.DataFrame(
    new_gene_info_rows,
    columns=[
        "gene_id",
        "gene_symbol",
        "melpomene_chromosome",
        "melpomene_scaffold",
        "melpomene_start",
        "melpomene_end",
        "erato_chromosome",
        "erato_scaffold",
        "erato_start",
        "erato_end",
        "erato_iou_score",
        "erato_containment_score",
    ],
)

gene_aligment_info_df.head()

,gene_id,gene_symbol,melpomene_chromosome,melpomene_scaffold,melpomene_start,melpomene_end,erato_chromosome,erato_scaffold,erato_start,erato_end,erato_iou_score,erato_containment_score
0,HMEL005427,NOS-B,0,67,69,6264,-1,-1,-1,-1,0.000000,0.000000
1,HMEL002216,Def,1,1,10505178,10506149,1,1,13904622,13904976,0.344130,0.952381
2,HMEL011647,TEP-B,1,1,11230994,11243629,1,1,14783054,14784415,0.043981,0.430657
3,HMEL011434,Wnt10,1,1,14208540,14229353,1,1,18432632,18457202,0.975259,0.975259
4,HMEL011436,Wnt6,1,1,14260513,14295660,1,1,18493665,18534835,0.954252,0.954252


In [84]:
gene_aligment_info_df.to_csv(DATA_DIR / "gene_alignment.csv")